<a href="https://colab.research.google.com/github/maggiecope/comp351-ai-project/blob/main/Maggie_Decision_Tree_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')


In [47]:
file_path = "Wait times with Weather (1).csv"


df = pd.read_csv(file_path, encoding='latin-1')

print("Dataset loaded!")
print("Shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
df.head()

Dataset loaded!
Shape: (339064, 26)

Column names:
['Land', 'Ride', 'Wait Time', 'Local Time', 'Day of Week', 'Date', 'tempmax', 'tempmin', 'temp', 'humidity', 'precip', 'windgust', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise', 'sunset', 'moonphase', 'conditions', 'description']


,Land,Ride,Wait Time,Local Time,Day of Week,Date,tempmax,tempmin,temp,humidity,...,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description
0,Pixar Pier,Toy Story Midway Mania!,40.0,2024-09-23 17:30:04,Monday,2024-09-23,78.4,62.4,67.9,80.5,...,5.6,210.0,18.0,8.0,10.0,2024-09-23 06:40:56,2024-09-23 18:46:15,0.69,Partially cloudy,Clearing in the afternoon.
1,Cars Land,Mater's Graveyard JamBOOree,30.0,2024-09-23 17:45:03,Monday,2024-09-23,78.4,62.4,67.9,80.5,...,5.6,210.0,18.0,8.0,10.0,2024-09-23 06:40:56,2024-09-23 18:46:15,0.69,Partially cloudy,Clearing in the afternoon.
2,Hollywood Land,Mickey's PhilharMagic,10.0,2024-09-23 17:45:03,Monday,2024-09-23,78.4,62.4,67.9,80.5,...,5.6,210.0,18.0,8.0,10.0,2024-09-23 06:40:56,2024-09-23 18:46:15,0.69,Partially cloudy,Clearing in the afternoon.
3,Pixar Pier,Jumpin' Jellyfish,20.0,2024-09-23 17:45:03,Monday,2024-09-23,78.4,62.4,67.9,80.5,...,5.6,210.0,18.0,8.0,10.0,2024-09-23 06:40:56,2024-09-23 18:46:15,0.69,Partially cloudy,Clearing in the afternoon.
4,Avengers Campus,WEB SLINGERS: A Spider-Man Adventure,45.0,2024-09-23 18:00:04,Monday,2024-09-23,78.4,62.4,67.9,80.5,...,5.6,210.0,18.0,8.0,10.0,2024-09-23 06:40:56,2024-09-23 18:46:15,0.69,Partially cloudy,Clearing in the afternoon.


Descision Tree

In [48]:

# Check dataset shape
print("\nDataset Shape:", df.shape)

# Check for missing values
print("\nMissing Values:")
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("No missing values!")

# Basic statistics
print("\nBasic Statistics:")
display(df.describe())


Dataset Shape: (339064, 26)

Missing Values:
No missing values!

Basic Statistics:


,Wait Time,tempmax,tempmin,temp,humidity,precip,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,moonphase
count,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000,339064.000000
mean,27.251177,74.027993,55.934031,63.797000,59.082741,0.009318,19.240843,9.736638,207.334571,1015.246004,36.919983,8.481364,193.952243,16.754742,7.194235,10.015543,0.475995
std,24.584424,8.859595,7.864218,7.257992,14.312078,0.084559,5.435196,2.780550,55.626174,3.539897,28.445428,1.547574,77.182473,6.658457,2.035255,0.278339,0.286508
min,5.000000,52.700000,39.900000,50.000000,12.600000,0.000000,9.200000,4.200000,3.700000,1006.300000,0.000000,2.000000,18.700000,1.700000,1.000000,10.000000,0.000000
25%,5.000000,67.000000,49.500000,58.200000,49.000000,0.000000,15.700000,8.200000,193.000000,1012.800000,13.100000,8.100000,131.700000,11.300000,6.000000,10.000000,0.240000
50%,20.000000,73.500000,55.300000,62.500000,57.900000,0.000000,19.000000,9.500000,211.600000,1014.800000,30.300000,9.100000,195.300000,16.800000,8.000000,10.000000,0.470000
75%,40.000000,80.300000,62.900000,68.900000,70.800000,0.000000,21.000000,10.900000,237.500000,1017.600000,58.600000,9.500000,260.600000,22.400000,9.000000,10.000000,0.730000
max,225.000000,107.500000,72.700000,86.900000,90.700000,1.701000,59.800000,24.500000,354.900000,1027.600000,100.000000,9.900000,331.100000,28.700000,10.000000,15.000000,0.980000


In [49]:
# Prepare Data

# convert main timestamp fields into datetime object

df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df["Local Time"] = pd.to_datetime(df["Local Time"], errors="coerce")

# Convert sunrise and sunset timestamps
df["sunrise"] = pd.to_datetime(df["sunrise"], errors="coerce")
df["sunset"] = pd.to_datetime(df["sunset"], errors="coerce")

# Preview the converted fields
df[["Date", "Local Time", "sunrise", "sunset"]].head()

,Date,Local Time,sunrise,sunset
0,2024-09-23,2024-09-23 17:30:04,2024-09-23 06:40:56,2024-09-23 18:46:15
1,2024-09-23,2024-09-23 17:45:03,2024-09-23 06:40:56,2024-09-23 18:46:15
2,2024-09-23,2024-09-23 17:45:03,2024-09-23 06:40:56,2024-09-23 18:46:15
3,2024-09-23,2024-09-23 17:45:03,2024-09-23 06:40:56,2024-09-23 18:46:15
4,2024-09-23,2024-09-23 18:00:04,2024-09-23 06:40:56,2024-09-23 18:46:15


In [50]:
# Split Data
trainX, testX, trainY, testY = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training samples: {trainX.shape[0]}")
print(f"Testing samples: {testX.shape[0]}")

Training samples: 271251
Testing samples: 67813


In [51]:
# Baseline Model
# Create and train model
dt_baseline = DecisionTreeRegressor(random_state=42)
dt_baseline.fit(trainX, trainY)

# Make predictions
train_pred = dt_baseline.predict(trainX)
test_pred = dt_baseline.predict(testX)

# Calculate metrics
train_r2 = r2_score(trainY, train_pred)
test_r2 = r2_score(testY, test_pred)
test_rmse = np.sqrt(mean_squared_error(testY, test_pred))
test_mae = mean_absolute_error(testY, test_pred)

print("BASELINE MODEL RESULTS:")
print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"Test RMSE: {test_rmse:.2f} minutes")
print(f"Test MAE: {test_mae:.2f} minutes")

BASELINE MODEL RESULTS:
Train R²: 1.0000
Test R²: 0.8161
Test RMSE: 10.49 minutes
Test MAE: 5.33 minutes


In [52]:
# Tuned Descision Tree
# Create tuned model (prevents overfitting)
dt_tuned = DecisionTreeRegressor(
    max_depth=15,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)

dt_tuned.fit(trainX, trainY)

# Make predictions
train_pred_tuned = dt_tuned.predict(trainX)
test_pred_tuned = dt_tuned.predict(testX)

# Calculate metrics
train_r2_tuned = r2_score(trainY, train_pred_tuned)
test_r2_tuned = r2_score(testY, test_pred_tuned)
test_rmse_tuned = np.sqrt(mean_squared_error(testY, test_pred_tuned))
test_mae_tuned = mean_absolute_error(testY, test_pred_tuned)

print("TUNED MODEL RESULTS:")
print(f"Train R²: {train_r2_tuned:.4f}")
print(f"Test R²: {test_r2_tuned:.4f}")
print(f"Test RMSE: {test_rmse_tuned:.2f} minutes")
print(f"Test MAE: {test_mae_tuned:.2f} minutes")

TUNED MODEL RESULTS:
Train R²: 0.7858
Test R²: 0.7564
Test RMSE: 12.08 minutes
Test MAE: 8.61 minutes


In [53]:
# Compare Models
comparison = pd.DataFrame({
    'Model': ['Baseline', 'Tuned'],
    'Test R²': [test_r2, test_r2_tuned],
    'Test RMSE': [test_rmse, test_rmse_tuned],
    'Test MAE': [test_mae, test_mae_tuned]
})

print("\nMODEL COMPARISON:")
print(comparison)


MODEL COMPARISON:
      Model   Test R²  Test RMSE  Test MAE
0  Baseline  0.816105  10.490452  5.329126
1     Tuned  0.756353  12.075046  8.607189


In [54]:
df["Local Time"] = pd.to_datetime(df["Local Time"], errors="coerce")
df["sunrise"] = pd.to_datetime(df["sunrise"], errors="coerce")
df["sunset"] = pd.to_datetime(df["sunset"], errors="coerce")

# Extract hour from 'Local Time'
df['hour'] = df['Local Time'].dt.hour

# Calculate time differences in minutes
df['minutes_since_sunrise'] = (df['Local Time'] - df['sunrise']).dt.total_seconds() / 60
df['minutes_until_sunset'] = (df['sunset'] - df['Local Time']).dt.total_seconds() / 60

print("Converted 'Local Time', 'sunrise', 'sunset' to datetime and created 'hour', 'minutes_since_sunrise', 'minutes_until_sunset' features.")
df[['Local Time', 'sunrise', 'sunset', 'hour', 'minutes_since_sunrise', 'minutes_until_sunset']].head()

Converted 'Local Time', 'sunrise', 'sunset' to datetime and created 'hour', 'minutes_since_sunrise', 'minutes_until_sunset' features.


,Local Time,sunrise,sunset,hour,minutes_since_sunrise,minutes_until_sunset
0,2024-09-23 17:30:04,2024-09-23 06:40:56,2024-09-23 18:46:15,17,649.133333,76.183333
1,2024-09-23 17:45:03,2024-09-23 06:40:56,2024-09-23 18:46:15,17,664.116667,61.200000
2,2024-09-23 17:45:03,2024-09-23 06:40:56,2024-09-23 18:46:15,17,664.116667,61.200000
3,2024-09-23 17:45:03,2024-09-23 06:40:56,2024-09-23 18:46:15,17,664.116667,61.200000
4,2024-09-23 18:00:04,2024-09-23 06:40:56,2024-09-23 18:46:15,18,679.133333,46.183333


In [55]:
categorical_features = ['Ride', 'Land', 'Day of Week', 'conditions']
numerical_features = [
    'tempmax', 'tempmin', 'temp', 'humidity', 'precip', 'windgust', 'windspeed',
    'winddir', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
    'solarenergy', 'uvindex', 'severerisk',
    'hour', 'minutes_since_sunrise', 'minutes_until_sunset'
]

# One-hot encode categorical features
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

# Select features for X
X = df_encoded[numerical_features + [col for col in df_encoded.columns if col.startswith(tuple(categorical_features)) and col not in categorical_features]]

# Define target variable y
y = df['Wait Time']

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("First 5 rows of X:")
display(X.head())
print("First 5 rows of y:")
display(y.head())

Shape of X: (339064, 62)
Shape of y: (339064,)
First 5 rows of X:


,tempmax,tempmin,temp,humidity,precip,windgust,windspeed,winddir,sealevelpressure,cloudcover,...,Day of Week_Monday,Day of Week_Saturday,Day of Week_Sunday,Day of Week_Thursday,Day of Week_Tuesday,Day of Week_Wednesday,conditions_Overcast,conditions_Partially cloudy,"conditions_Rain, Overcast","conditions_Rain, Partially cloudy"
0,78.4,62.4,67.9,80.5,0.0,17.7,7.8,243.8,1009.7,47.1,...,True,False,False,False,False,False,False,True,False,False
1,78.4,62.4,67.9,80.5,0.0,17.7,7.8,243.8,1009.7,47.1,...,True,False,False,False,False,False,False,True,False,False
2,78.4,62.4,67.9,80.5,0.0,17.7,7.8,243.8,1009.7,47.1,...,True,False,False,False,False,False,False,True,False,False
3,78.4,62.4,67.9,80.5,0.0,17.7,7.8,243.8,1009.7,47.1,...,True,False,False,False,False,False,False,True,False,False
4,78.4,62.4,67.9,80.5,0.0,17.7,7.8,243.8,1009.7,47.1,...,True,False,False,False,False,False,False,True,False,False


First 5 rows of y:


,Wait Time
0,40.0
1,30.0
2,10.0
3,20.0
4,45.0


In [56]:
trainX, testX, trainY, testY = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training samples: {trainX.shape[0]}")
print(f"Testing samples: {testX.shape[0]}")

Training samples: 271251
Testing samples: 67813


In [57]:
# Baseline Model
# Create and train model
dt_baseline = DecisionTreeRegressor(random_state=42)
dt_baseline.fit(trainX, trainY)

# Make predictions
train_pred = dt_baseline.predict(trainX)
test_pred = dt_baseline.predict(testX)

# Calculate metrics
train_r2 = r2_score(trainY, train_pred)
test_r2 = r2_score(testY, test_pred)
test_rmse = np.sqrt(mean_squared_error(testY, test_pred))
test_mae = mean_absolute_error(testY, test_pred)

print("BASELINE MODEL RESULTS:")
print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"Test RMSE: {test_rmse:.2f} minutes")
print(f"Test MAE: {test_mae:.2f} minutes")

BASELINE MODEL RESULTS:
Train R²: 1.0000
Test R²: 0.8161
Test RMSE: 10.49 minutes
Test MAE: 5.33 minutes


In [58]:
# Tuned Descision Tree
# Create tuned model (prevents overfitting)
dt_tuned = DecisionTreeRegressor(
    max_depth=15,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)

dt_tuned.fit(trainX, trainY)

# Make predictions
train_pred_tuned = dt_tuned.predict(trainX)
test_pred_tuned = dt_tuned.predict(testX)

# Calculate metrics
train_r2_tuned = r2_score(trainY, train_pred_tuned)
test_r2_tuned = r2_score(testY, test_pred_tuned)
test_rmse_tuned = np.sqrt(mean_squared_error(testY, test_pred_tuned))
test_mae_tuned = mean_absolute_error(testY, test_pred_tuned)

print("TUNED MODEL RESULTS:")
print(f"Train R²: {train_r2_tuned:.4f}")
print(f"Test R²: {test_r2_tuned:.4f}")
print(f"Test RMSE: {test_rmse_tuned:.2f} minutes")
print(f"Test MAE: {test_mae_tuned:.2f} minutes")

TUNED MODEL RESULTS:
Train R²: 0.7858
Test R²: 0.7564
Test RMSE: 12.08 minutes
Test MAE: 8.61 minutes
